# FiLM: Feature-wise Linear Modulation

In [ ]:
from models import *
from utils import *

In [ ]:
class FiLMBlock(nn.Module):
    def __init__(self):
        super(FiLMBlock, self).__init__()
        
    def forward(self, x, gamma, beta):
    
        x = gamma * x + beta
        
        return x

class FiLMFusionNet(nn.Module):
    def __init__(self, eeg_dim, eeg_emb_dim,pos, phy_dim, phy_emb_dim, phy_feat_dim, eeg_feat_dim=5, n_class=2):
        
        self.eeg_attention = RegionRNN_DEAP(eeg_emb_dim//2, 1, eeg_feat_dim, f_dim=eeg_feat_dim)
        self.phy_attention = SimpleRNN_DEAP(phy_emb_dim, 1, phy_feat_dim, f_dim=phy_feat_dim)

        self.FiLM_GenEEG = nn.Linear(eeg_emb_dim, 2)
        self.FiLM_GenPhy = nn.Linear(phy_emb_dim, 2)
        
        self.Dense_EEG = nn.Sequential(
            nn.Linear(eeg_emb_dim, 64),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(64)
        )
        
        self.Dense_Phy = nn.Sequential(
            nn.Linear(phy_emb_dim, phy_emb_dim),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(phy_emb_dim)
        )
        
        self.bn_eeg = nn.BatchNorm1d(eeg_emb_dim)
        self.bn_phy = nn.BatchNorm1d(phy_emb_dim)
        
        self.Classifier = nn.Sequential(
            nn.Linear(eeg_emb_dim+phy_emb_dim, 64),
            nn.Dropout(0.25),
            nn.ReLU(True),
            nn.Linear(64, n_class),
            nn.Softmax(dim=1)
        )
        
        self.Discriminator = nn.Sequential(
            GradientReversal(),
            nn.Linear(eeg_emb_dim+phy_emb_dim, 64),
            nn.Dropout(0.25),
            nn.ReLU(True),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
        
        self.film = FiLMBlock()
    def forward(self, x_eeg, x_phy):
        b_size = x_eeg.shape[0]
        
        spatial_eeg = x_eeg.transpose(1,2)
        feat_eeg = self.bn_eeg(self.eeg_attention(spatial_eeg))
        
        spatial_phy = x_phy.transpose(1,2)
        feat_phy = self.bn_phy(self.phy_attention(spatial_phy))
        
        film_eeg = self.FiLM_GenEEG(feat_eeg)
        film_phy = self.FiLM_GenPhy(feat_phy)
        
        feat_eeg = self.film(feat_eeg, film_phy[:,0].unsqueeze(1), film_phy[:,1].unsqueeze(1))
        feat_phy = self.film(feat_phy, film_eeg[:,0].unsqueeze(1), film_eeg[:,1].unsqueeze(1))
        
        feat_eeg = self.Dense_EEG(feat_eeg)
        feat_phy = self.Dense_Phy(feat_phy)
        
        out = torch.cat([feat_eeg, feat_phy], dim=1)
        return out

In [ ]:
'''Parameters'''
batch_size = 64
n_epoch = 500

In [ ]:
def train_model(feat_eeg, feat_phy, label, participant, path_results, training_info, n_class, pos):
    EEG = EEGPhiDataset(label=label, eeg=feat_eeg, phi=feat_phy)
    Tot = {}
    session = np.load('dataset/seed_iv/session.npy')
    for p in tqdm(np.unique(participant)):
        idx = np.argwhere(participant==p).squeeze()
        np.random.shuffle(idx)
        id_train = idx[:int(0.8*len(idx))]
        id_test = idx[int(0.8*len(idx)):]
        np.random.shuffle(id_train)
        np.random.shuffle(id_test)
        Test = Subset(EEG, id_test)
        Train = Subset(EEG, id_train)
        
        Trainloader = DataLoader(Train, batch_size=batch_size, shuffle=False)
        Testloader = DataLoader(Test, batch_size=batch_size, shuffle=False)
        
        n_chan = feat_eeg.shape[2]
        f_dim_eeg = feat_eeg.shape[1]
        
        phy_dim = feat_phy.shape[2]
        f_dim_phy = feat_phy.shape[1]
        
        net = FiLMFusionNet(eeg_dim=n_chan, eeg_emb_dim=64, eeg_feat_dim=f_dim_eeg, pos=pos, phy_dim=phy_dim, phy_emb_dim=16, phy_feat_dim=f_dim_phy, n_class=n_class).cuda()
        
        optimizer = optim.SGD(net.parameters(), lr=0.0005, momentum=0.9, weight_decay=0.005)
        
        res = []
        for epoch in range(n_epoch):
            
            running_loss = 0.0
            evaluation = []
            t_cycle = iter(cycle(Testloader))
            for i, data in enumerate(Trainloader, 0):
                # get the inputs; data is a list of [inputs, labels]
                inputs_eeg_source, inputs_phy_source, labels = data
                del data
                data = next(t_cycle)
                inputs_eeg_test, inputs_phy_test, _ = data
                del data
                
                domain = torch.cat([torch.ones(inputs_eeg_source.shape[0]),
                                   torch.zeros(inputs_eeg_test.shape[0])]).cuda()
                inputs_eeg = torch.cat([inputs_eeg_source, inputs_eeg_test])
                inputs_phy = torch.cat([inputs_phy_source, inputs_phy_test])
                
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward + backward + optimize
                feat_ = net(inputs_eeg.to(torch.float32).cuda(), inputs_phy.to(torch.float32).cuda())
                
                domain_pred = net.Discriminator(feat_).squeeze()
                
                feat_ = net(inputs_eeg.to(torch.float32).cuda(), inputs_phy.to(torch.float32).cuda())
                loss = torch.nn.functional.binary_cross_entropy(domain_pred, domain)
                outputs = net.Classifier(feat_[:inputs_eeg_source.shape[0]])

                label_loss = torch.nn.functional.cross_entropy(outputs, labels.to(torch.long).cuda())
                loss = label_loss
                loss.backward()
                optimizer.step()

                _, predicted = torch.max(outputs, 1)
                num_of_true = torch.sum(predicted.detach().cpu()==labels).numpy()
                mean = num_of_true/labels.shape[0]
                running_loss = label_loss.item()
                evaluation.append(mean)
            running_loss = running_loss/(i+1)
            running_acc = sum(evaluation)/len(evaluation)
            
            validation_loss = 0.0
            validation_acc = 0.0
            evaluation = []
            for i, data in enumerate(Testloader, 0):
                inputs_eeg, inputs_phy, labels = data
                del data
                feat_ = net(inputs_eeg.to(torch.float32).cuda(), inputs_phy.to(torch.float32).cuda())
                outputs = net.Classifier(feat_)
                loss = torch.nn.functional.cross_entropy(outputs, labels.cuda())
                validation_loss += loss.item()
                
                _, predicted = torch.max(outputs, 1)
                num_of_true = torch.sum(predicted.detach().cpu()==labels).numpy()
                evaluation.append(num_of_true/labels.shape[0])
            validation_loss = validation_loss/(i+1)
            validation_acc = sum(evaluation)/len(evaluation)
            
            res.append((running_loss, running_acc, validation_loss, validation_acc))
        Tot['participant_'+str(p)] = np.asarray(res)
        np.save(os.path.join(path_results, training_info), Tot)

## PhyDAA Dataset

In [ ]:
'''Load File'''
label = np.load('dataset/phydaa/label.npy').astype(int)
n_class = len(np.unique(label))
participant = np.load('dataset/phydaa/participant.npy')
elec_pos = np.load('information/phydaa_eeg.npy')

In [ ]:
train_model(feat_eeg=np.load('dataset/phydaa/feat_eeg.npy'), feat_phy=np.expand_dims(np.load('dataset/phydaa/feat_phy.npy'),1), label=label, 
            participant=participant, path_results='res/dep/', training_info='phydaa_FiLM', n_class=n_class, pos=elec_pos)

## SEED IV

In [ ]:
'''Load File'''
label = np.load('dataset/seed_iv//label.npy').astype(int)
n_class = len(np.unique(label))
participant = np.load('dataset/seed_iv/participant.npy')
elec_pos = np.load('information/seed_iv_eeg.npy')

In [ ]:
phy_feat = np.load('dataset/seed_iv/feat_phy.npy')
phy_feat[:, :12] = (phy_feat[:, :12] - phy_feat[:, :12].min())/np.max(phy_feat[:, :12] - phy_feat[:, :12].min()) 
phy_feat[:, 12:16] = (phy_feat[:, 12:16] - phy_feat[:, 12:16].min())/np.max(phy_feat[:, 12:16] - phy_feat[:, 12:16].min()) 
phy_feat[:, 16:18] = (phy_feat[:, 16:18] - phy_feat[:, 16:18].min())/np.max(phy_feat[:, 16:18] - phy_feat[:, 16:18].min()) 
phy_feat[:, 18:22] = (phy_feat[:, 18:22] - phy_feat[:, 18:22].min())/np.max(phy_feat[:, 18:22] - phy_feat[:, 18:22].min())

train_model(feat_eeg=np.load('dataset/seed_iv/feat_eeg.npy'), feat_phy=np.expand_dims(phy_feat,1)[:, :, :22], label=label,
            participant=participant, path_results='res/dep', training_info='seed_iv_FiLM_rnn', n_class=n_class, pos=elec_pos)

## DEAP

In [ ]:
'''Load File'''
label = np.load('dataset/deap/label.npy') - 4.5
label = (label[:, 0]>0).astype(int) + 2*(label[:, 1]>0).astype(int)
n_class = len(np.unique(label))
participant = np.load('dataset/deap/participant.npy')

In [ ]:
train_model(feat_eeg=np.load('dataset/deap/feat_eeg.npy'), feat_phy=np.load('dataset/deap/feat_phy.npy'), label=label, 
            participant=participant, path_results='res/dep/', training_info='deap_attention_rnn', n_class=n_class, pos=None)

In [ ]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)